In [1]:
import cv2 as cv
from cv2 import VideoCapture

In [2]:
from view_controller import ViewController

cap = VideoCapture("worms.avi")

sim = ViewController(cap, color_conversion=cv.COLOR_BGR2GRAY)

while sim.next_frame():
    sim.visualize_world()
    sim.move_position(5, 5)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

cv.destroyAllWindows()